In [2]:
login_url = 'https://secure.indeed.com/account/login'

In [3]:
data = {
        'action':'login',
        '__email':'vicyan614@gmail.com',
        '__password':'yhc900614',
        'remember':'1',
        'hl':'en',
        'continue':'/account/view?hl=en',
       }

In [4]:
header = {'User_Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'}

In [5]:
def periodfinder(content,class_info):
    if content.find('p',class_=class_info) is not None:
            period = i.find('p',class_=class_info).text.split(' to ')
            #print (period)   # AA BB 1, AA PRESENT 2 A PRESENT 3 A BB 4 AA B 5
            if len(period) == 2:
                if period[1] == 'Present':
                    try:
                        period[1] = datetime.today().strftime('%B %Y')
                    except:
                        period[1] = datetime.today().strftime('%b %Y')
                if len(period[0]) > 4 and len(period[1]) > 4:
                    #type AA BB
                    try:
                        prd = (datetime.strptime(period[1],'%B %Y').date()-\
                        datetime.strptime(period[0],'%B %Y').date())
                    except:
                        prd = (datetime.strptime(period[1],'%b %Y').date()-\
                        datetime.strptime(period[0],'%b %Y').date())
                    
                    YOE = int(abs(round(prd.days/365,0)))
                elif len(period[0]) > 4 and len(period[1]) < 5:
                    #type AA B
                    YOE = int(period[1])-int(period[0].split()[1])
                elif len(period[0]) < 5 and len(period[1]) > 4:
                    #type A BB
                    YOE = int(period[1].split()[1])-int(period[0])
                elif len(period[0]) < 5 and len(period[1]) < 5:
                    #type A B
                    YOE = int(period[1])-int(period[0])
            else:
                YOE = 1
            if YOE < 1:
                YOE = 'less than 1 year'
    else:
        YOE = 'No lenth info'
    return YOE

In [8]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re
#start session s
s = requests.session()
#login with login and password
r = s.post(login_url,data = data)
#get resume page1 
rq= s.get('https://www.indeed.com/resumes?q=Jasmeet%20Singh&l=GTA%2C%20ON&from=mrs&subfrom=1_10')
url = []
soup = ''
all_content=[]
soup = BeautifulSoup(rq.text,'html.parser')
content = soup.find_all('a', class_= 'app_link')
#get urls in a page
for c in content:
    full = 'https://www.indeed.com'+c.attrs['href']
    url.append(full)
cnt = len(content)
#open each url and look for contents
for j in range(0,cnt):
    r = s.get(url[j],headers=header)
    soup = BeautifulSoup(r.text,'html.parser')
    #Resume name
    name = soup.find('h1').text
    print(name)
    #experience 
    experience = soup.find_all('div',class_=re.compile(r'work-experience-section( \w)*'))
    for i in experience:
        if i.find('p',class_='work_title title') is None:
            jobtitle = 'No Job Title'
        else:
            jobtitle = i.find('p',class_='work_title title').text
        if i.find('div',class_='work_company') is None:
            comp_lo = 'No Company'
        else:
            comp_lo = i.find('div',class_='work_company').text.split(' - ')
        if len(comp_lo) == 2:
            company,location = i.find('div',class_='work_company').text.split(' - ')
        elif len(comp_lo) ==1 :
            company = i.find('div',class_='work_company').text
        else:
            company = 'No Company'
        if i.find('p',class_='work_dates') is not None:
            YOE = 0
            period = i.find('p',class_='work_dates').text.split(' to ')
            #print (period)   # AA BB 1, AA PRESENT 2 A PRESENT 3 A BB 4 AA B 5
            if len(period) == 2:
                if period[1] == 'Present':
                    try:
                        period[1] = datetime.today().strftime('%B %Y')
                    except:
                        period[1] = datetime.today().strftime('%b %Y')
                if len(period[0]) > 4 and len(period[1]) > 4:
                    #type AA BB
                    try:
                        prd = (datetime.strptime(period[1],'%B %Y').date()-\
                        datetime.strptime(period[0],'%B %Y').date())
                    except:
                        prd = (datetime.strptime(period[1],'%b %Y').date()-\
                        datetime.strptime(period[0],'%b %Y').date())
                    
                    YOE = int(abs(round(prd.days/365,0)))
                elif len(period[0]) > 4 and len(period[1]) < 5:
                    #type AA B
                    YOE = int(period[1])-int(period[0].split()[1])
                elif len(period[0]) < 5 and len(period[1]) > 4:
                    #type A BB
                    YOE = int(period[1].split()[1])-int(period[0])
                elif len(period[0]) < 5 and len(period[1]) < 5:
                    #type A B
                    YOE = int(period[1])-int(period[0])
            else:
                YOE = 1
            if YOE < 1:
                YOE = 'less than 1 year'
        else:
            YOE = 'No lenth of work info'
        jobdesc = i.find('p',class_='work_description')
        if jobdesc is None:
            desc = ''
        else:
            desc = jobdesc.text
        print(jobtitle,'\n',company,location,'\n',desc,'\n',YOE,'year')
    #break
        #education education-section last
    educations = soup.find_all('div',class_=re.compile(r'education-section( \w)*'))
    for e in educations:
        if e.find('p', class_ = 'edu_title') is None:
            edu_title = 'No Education Information'
        else:
            edu_title = e.find('p', class_ = 'edu_title').text
        if e.find('div', class_='edu_school') is None:
            edu_school = 'No School Information'
        else:
            edu_school = e.find('div',class_='edu_school').text
        if e.find('div',class_='inline-block') is None:
            edu_location = 'No Education Location Info'
        else:
            edu_location = e.find('div',class_='inline-block').text
        if e.find('p',class_='edu_dates') is not None:
            YOEdu = 0
            edu_period = e.find('p',class_='edu_dates').text.split(' to ')
            print (period)   # AA BB 1, AA PRESENT 2 A PRESENT 3 A BB 4 AA B 5
            if len(edu_period) == 2:
                if edu_period[1] == 'Present':
                    try:
                        edu_period[1] = datetime.today().strftime('%B %Y')
                    except:
                        edu_period[1] = datetime.today().strftime('%b %Y')
                if len(edu_period[0]) > 4 and len(edu_period[1]) > 4:
                    #type AA BB
                    try:
                        edu_prd = (datetime.strptime(edu_period[1],'%B %Y').date()-\
                        datetime.strptime(edu_period[0],'%B %Y').date())
                    except:
                        edu_prd = (datetime.strptime(edu_period[1],'%b %Y').date()-\
                        datetime.strptime(period[0],'%b %Y').date())
                    
                    YOEdu = int(abs(round(edu_prd.days/365,0)))
                elif len(edu_period[0]) > 4 and len(edu_period[1]) < 5:
                    #type AA B
                    YOEdu = int(edu_period[1])-int(edu_period[0].split()[1])
                elif len(edu_period[0]) < 5 and len(edu_period[1]) > 4:
                    #type A BB
                    YOEdu = int(edu_period[1].split()[1])-int(edu_period[0])
                elif len(period[0]) < 5 and len(period[1]) < 5:
                    #type A B
                    YOEdu = int(edu_period[1])-int(edu_period[0])
            else:
                YOEdu = 1
            if YOEdu < 1:
                YOEdu = 'less than 1 year'
        else:
            YOEdu = 'No lenth of Edu info'
        print(edu_title,'\n',edu_school,edu_location,'\n',YOEdu,'year')
        #Skills
    skill_set = soup.find_all('span',class_='skill-text')
    for skills in skill_set:
        skill = skills.text
        print(skill)
        #Certifications/Licenses
    
    cert = soup.find_all('div',class_=re.compile(r'certification-section( \w)*'))
    for c in cert:
        print (c.text)
    break
        #Adwards
        
        #Additional Information

Jasmeet Singh
security mobile offiicer 
 G4S Toronto, ON 
  
 2 year
Quality Inspector 
 Martinrea Alfield Toronto, ON 
 with TRIGO, The Pic Group) Co-op • Collect and Record accurate data regarding defects • Tag and isolate defective and good material • Provide description and justification of defective material • Perform visual, mechanical and functional verification of parts to ensure compliance to set specific and quality standard • Checking the parts on the fixture 
 less than 1 year year
Graduate Engineer Trainee 
 International Tractors Limited Toronto, ON 
 • Performed process audit on pre-paint and post-paint tractor assembly line by visual inspection • Ensured adherence of process audit on differential assembly, gear box assembly and hydraulic assembly by visual method and using various equipment like Vernier caliper, micrometer and height gauge • Performed torque audit on the complete tractor assembly line by using torque wrench • Performed product audit (gate audit) after f

In [41]:
for e in experience:
    print (periodfinder(e,'work_dates'))

1
1
1


In [8]:
    educations = soup.find_all('div',class_=re.compile(r'education-section( \w)*'))
    for e in educations:
        if e.find('p', class_ = 'edu_title') is None:
            edu_title = 'No Education Information'
        else:
            edu_title = e.find('p', class_ = 'edu_title').text
        if e.find('div', class_='edu_school') is None:
            edu_school = 'No School Information'
        else:
            edu_school = e.find('div',class_='edu_school').text

In [18]:
a = e.find('div',class_='edu_school')

In [19]:
a.text

'Punjab Technical University'

In [17]:
e.find('div',class_='inline-block')

AttributeError: 'NoneType' object has no attribute 'text'